In [1]:
from data_prep_hybrid import hybrid_transformer_database
from eq_data_loader import get_eq_data
import json
import tensorflow as tf
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

2024-05-06 16:31:18.848643: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-06 16:31:18.884505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 16:31:18.884552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 16:31:18.885888: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 16:31:18.892421: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-06 16:31:18.893439: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
tf.__version__

'2.15.0'

In [3]:
data_config = json.load(open('data_config.json', 'r'))
data = get_eq_data(data_config['correlation_thresh'])
xtrain, xtest, ytrain, ytest, train_index, test_index = hybrid_transformer_database(data = data, timestep = 16, lag = 5, lagSD = 5, test_size= 0.2, purge_size = 30)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

[*********************100%%**********************]  50 of 50 completed


(1974, 16, 20) (501, 16, 20) (1974, 10) (501, 10)


In [4]:
from hybrid_transformer_model import HybridTransformer_Portfolio

Dropout = 0.05
LearningRate = 0.01
Epochs = 1000
Alpha = 0.005
DF = 4
BatchSize = 16
lb = 0
ub = 0.3
timestep = 16

priceData = tf.convert_to_tensor(data.loc[train_index, :].values, np.float32)
model = HybridTransformer_Portfolio(xtrain.shape[1], xtrain.shape[2], outputShape = ytrain.shape[1],  headsAttention = 4, dropout= Dropout, learningRate = LearningRate, priceData = priceData, lb= 0.01, ub = 0.2)
weights = model.allocation_hybrid(xtrain, ytrain, Epochs = Epochs, BatchSize = BatchSize)


ValueError: in user code:

    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py", line 1154, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/optimizers/optimizer.py", line 544, in minimize
        self.apply_gradients(grads_and_vars)
    File "/home/codespace/.python/current/

In [ ]:
tf.__version__

'2.15.0'